In [1]:
import cv2
import numpy as np
import scipy
import matplotlib.pylab as plt
from skimage import io

In [ ]:
img = cv2.imread('Template-1.png',1)
cap = cv2.VideoCapture('left_output-1.avi')

detector = cv2.SIFT_create()
matching = cv2.BFMatcher()
min_match_number = 10
img_gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)


while cap.isOpened():
    ret,frame = cap.read()
    ret,new = cap.read()
    cap_frame = frame.copy()
    frame_gray = cv2.cvtColor(cap_frame,cv2.COLOR_BGR2GRAY)
    wide = cap.get(3)
    high = cap.get(4)
    center_x = wide/2
    center_y = high/2
    if ret :
        img_kpts,img_desc = detector.detectAndCompute(img_gray,None)
        query_kpts,query_desc = detector.detectAndCompute(frame_gray,None)
        match = matching.knnMatch(img_desc,query_desc,k = 2)
        goodmatch = list()
        for m,n in match :
            if m.distance < 0.65*n.distance :
                goodmatch.append(m)
        if len(goodmatch) > min_match_number :
            src_pts = np.float32([ img_kpts[m.queryIdx].pt for m in goodmatch]).reshape(-1,1,2)
            dst_pts = np.float32([ query_kpts[m.trainIdx].pt for m in goodmatch]).reshape(-1,1,2)
            src_pts,dst_pts = np.float32((src_pts,dst_pts))
            H,status=cv2.findHomography(src_pts,dst_pts,cv2.RANSAC,0.5)
            h,w=img.shape[:2]
            Border=np.float32([[[0,0],[0,h],[w,h],[w,0]]]).reshape(-1,1,2)
            BoxBorder=cv2.perspectiveTransform(Border,H)
            x_axis = BoxBorder[0][0]
            y_axis = BoxBorder[1][0]
            z1 = BoxBorder[0]
            z2 = BoxBorder[1]
            z_axis = z2[0]-z1[0]
            detect_img = cv2.polylines(frame,[np.int32(BoxBorder)],True,(0,0,255),2)
            cv2.putText(frame,("X: {}, Y: {} , Z:{}".format(int(x_axis[0]-center_x),int(y_axis[1]-center_y),int(z_axis[1]))),(10,frame.shape[0]-10),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2,cv2.LINE_AA,False)
            cv2.imshow('result',detect_img)

        if cv2.waitKey(int(1000/30)) & 0xFF == ord('q') :
            break       
    else :
        break
cap.release()
cv2.destroyAllWindows()          

: 